# Setting

In [14]:
import logging
import os
import pdb
import math
import glob
import random
import time
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.preprocessing import LabelEncoder

from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split

import torchvision.transforms as T

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.nn.parameter import Parameter

from tqdm import tqdm
import sys

import copy
from scipy import spatial
import csv

from perceiver import crop, patchify, get_patch_coords, ImageDataset, PerceiverBlock, Perceiver

In [15]:
def seed_everything(seed):
    torch.manual_seed(seed) #torch를 거치는 모든 난수들의 생성순서를 고정한다
    torch.cuda.manual_seed(seed) #cuda를 사용하는 메소드들의 난수시드는 따로 고정해줘야한다 
    torch.cuda.manual_seed_all(seed)  # if use multi-GPU
    torch.backends.cudnn.deterministic = True #딥러닝에 특화된 CuDNN의 난수시드도 고정 
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed) #numpy를 사용할 경우 고정
    random.seed(seed) #파이썬 자체 모듈 random 모듈의 시드 고정
seed_everything(42)

# Load Model, DataLoader

In [16]:
root_dir = '/home/Minju/Perceiver/model/'
loader_dir = '/home/Minju/Perceiver/loader/'

batch_size = 32

In [17]:
input_models = []
valid_loaders = []
for i in range (6):
    text_model = torch.load(root_dir + f'text_model_{i+1}.pkl')
    input_models.append(text_model)
    print(f"Text model {i+1}번 불러오기 완료.")

    with open(loader_dir+f'text_val_loader_{i+1}.pkl', 'rb') as f:
        loaded_valid_dataset = pickle.load(f)
    
    valid_loader = DataLoader(loaded_valid_dataset, batch_size=batch_size, shuffle=False)
    valid_loaders.append(valid_loader)
    print(f"Text val. loader {i}번 불러오기 완료.")

# for i in range(6):
#     img_model = torch.load(root_dir + f'image_model_{i}.pkl')
#     input_models.append(img_model)
#     print(f"Image model {i}번 불러오기 완료.")

#     with open(loader_dir+f'image_val_loader_{i}.pkl', 'rb') as f:
#         loaded_valid_dataset = pickle.load(f)

#     valid_loader = DataLoader(loaded_valid_dataset, batch_size=batch_size, shuffle=False)
#     valid_loaders.append(valid_loader)
#     print(f"Image val. loader {i}번 불러오기 완료.")

Text model 1번 불러오기 완료.
Text val. loader 0번 불러오기 완료.
Text model 2번 불러오기 완료.
Text val. loader 1번 불러오기 완료.
Text model 3번 불러오기 완료.
Text val. loader 2번 불러오기 완료.
Text model 4번 불러오기 완료.
Text val. loader 3번 불러오기 완료.
Text model 5번 불러오기 완료.
Text val. loader 4번 불러오기 완료.
Text model 6번 불러오기 완료.
Text val. loader 5번 불러오기 완료.


/tmp/ipykernel_145447/2580130533.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_model = torch.load(root_dir + f'text_model_{i+1}.pkl')


In [18]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Define ModelDiff

In [19]:
num_classes = -1
lr = 0.1
batch_size = 32
val_batch_size = 100
workers = 24
weight_decay = 4e-5
dataset_name = ''
train_path = ''
val_path = ''
cuda = True
seed = 1
epochs = 160
restore_epoch = 0
save_folder = ''
load_folder = ''
one_shot_prune_perc = 0.5
mode = ''
logfile = ''
initial_from_task = ''

In [20]:
DATASETS = [
    'Opinion','Art & Design','Television',
    'Music','Travel','Real Estate',
    'Books','Theater','Health',
    'Sports','Science','Food',
    'Fashion & Style','Movies','Technology',
    'Dance', 'Media', 'Style'
]

In [21]:
epsilon = 0.1
max_iterations = 100

In [22]:
target_id = 0

# 유사도검색

task_id 책정방식: \
0~5 : text modality \
6~11: image modality

## Approach 1: 특정 input data로 유사도 검증

In [23]:
def compute_ddv_cos(model1, model2, inputs):
    global outputs
    global outputs2
        
    with torch.no_grad():
        dists = []
        outputs = model1(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs = int(len(list(inputs)) / 2)
        for i in range(n_pairs):
            ya = outputs[i]
            yb = outputs[i + n_pairs]
            dist = spatial.distance.cosine(ya, yb)
            dists.append(dist)

        dists2 = []
        outputs2 = model2(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs2 = int(len(list(inputs)) / 2)
        for i in range(n_pairs2):
            ya = outputs2[i]
            yb = outputs2[i + n_pairs]
            dist = spatial.distance.cosine(ya, yb)
            dists2.append(dist)
    return np.array(dists), np.array(dists2)

In [24]:
def compute_ddv_euc(model1, model2, inputs):
    global outputs
    global outputs2
        
    with torch.no_grad():
        dists = []
        outputs = model1(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs = int(len(list(inputs)) / 2)
        for i in range(n_pairs):
            ya = outputs[i]
            yb = outputs[i + n_pairs]
            dist = spatial.distance.euclidean(ya, yb) # dist = spatial.distance.cosine(ya, yb)
            dists.append(dist)

        dists2 = []
        outputs2 = model2(torch.Tensor(inputs).cuda()).to('cpu').tolist()
        n_pairs2 = int(len(list(inputs)) / 2)
        for i in range(n_pairs2):
            ya = outputs2[i]
            yb = outputs2[i + n_pairs]
            dist = spatial.distance.euclidean(ya, yb) # dist = spatial.distance.cosine(ya, yb)
            dists2.append(dist)
    return np.array(dists), np.array(dists2)

In [25]:
##### compute_similarity #####
def compute_sim_cos(ddv1, ddv2):
    return spatial.distance.cosine(ddv1, ddv2)    

In [27]:
ddvcc_list = []
ddvec_list = []

for task_id in range(6):
    if task_id == target_id:
        continue


    batch = next(iter(valid_loaders[target_id]))  
    print(batch)
    inputs = batch['input_ids']  # 첫 번째 배치의 입력만 사용

    ddv1, ddv2 = compute_ddv_cos(input_models[target_id], input_models[task_id], inputs)
    ddv_distance = compute_sim_cos(ddv1, ddv2)
    print('DDV cos-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
    ddvcc_list.append(ddv_distance)

    # DDV-EC
    ddv1, ddv2 = compute_ddv_euc(input_models[target_id], input_models[task_id], inputs)
    ddv_distance = compute_sim_cos(ddv1, ddv2)
    print('DDV euc-cos [%d => %d] %.5f'%(task_id, target_id, ddv_distance))
    ddvec_list.append(ddv_distance)

TypeError: 'DataLoader' object is not subscriptable

## Approach 2: latent vector로 유사도 검증